In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm

warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [ ]:

def clean_weather_data(weather_filenm:str, method:str='linear', gap_limit:int=None, limit_direction:str='forward', save_filenm=None):
    """
    Assumes weather_filenm is of the format ASHRAE provided
    
    :param weather_filenm: 
    :param method : {‘linear’, ‘time’, ‘index’, ‘values’, ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘barycentric’, ‘krogh’, ‘polynomial’, ‘spline’, ‘piecewise_polynomial’, ‘from_derivatives’, ‘pchip’, ‘akima’} 
    :param gap_limit: Maximum number of consecutive hours to fill. Must be greater than 0.
    :param limit_direction: forward/backward/both
    :return: 
    """
    df_weather_dtypes = {'site_id': np.int8, 'air_temperature': np.float32, 'cloud_coverage': np.float32, 'dew_temperature': np.float32,
                     'precip_depth_1_hr': np.float32, 'sea_level_pressure': np.float32, 'wind_direction': np.float32, 'wind_speed': np.float32}

    weather_df = pd.read_csv(weather_filenm, dtype=df_weather_dtypes, parse_dates=['timestamp'])
    grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=method, limit=gap_limit, limit_direction=limit_direction))
    
    if 'cloud_coverage' in grouped_weather_df.columns:
        grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
        
    grouped_weather_df.reset_index(inplace=True)
    if save_filenm!=None:
        grouped_weather_df.to_csv(save_filenm)

    return grouped_weather_df


data_folder = './input/ashrae-energy-prediction/'
weather_train_filenm = f'{data_folder}/weather_train.csv'

interp_weather_train_filenm = f'{data_folder}/fully_interpolated_weather_train.csv'
grouped_weather_train = clean_weather_data(weather_train_filenm, method='linear', gap_limit=None, save_filenm=interp_weather_train_filenm)

partially_interp_weather_train_filenm = f'{data_folder}/partially_interpolated_weather_train.csv'
grouped_weather_train_with_gap_limit = clean_weather_data(weather_train_filenm, method='linear', gap_limit=3, save_filenm=partially_interp_weather_train_filenm)

print(grouped_weather_train.head(100))
print(partially_interp_weather_train_filenm.head(100))

In [2]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.int32, 'year_built': np.float32, 'floor_count': np.float32},
    'weather' : {'site_id': np.int8, 'air_temperature': np.float32, 'cloud_coverage': np.float32, 'dew_temperature': np.float32,
                     'precip_depth_1_hr': np.float32, 'sea_level_pressure': np.float32, 'wind_direction': np.float32, 'wind_speed': np.float32}
}

file_loc = {}    
for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
    for name in ['building_metadata','weather_train','weather_test','train','test']:
        if path.exists(dir_path + name + '.csv'):
            file_loc[name]= dir_path + name + '.csv'
    
    building = pd.read_csv(file_loc['building_metadata'], dtype=file_dtype['building_metadata'])
    weather_train = pd.read_csv(file_loc['weather_train'], dtype=file_dtype['weather'])
    weather_test = pd.read_csv(file_loc['weather_test'], dtype=file_dtype['weather'])
    train = pd.read_csv(file_loc['train'], dtype=file_dtype['train'])
    test = pd.read_csv(file_loc['test'], dtype=file_dtype['test'])

train = train.merge(building, on='building_id', how='left')
test = test.merge(building, on='building_id', how='left')
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')

del weather_train, weather_test
gc.collect()

109

In [3]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [4]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [5]:
class SetCatTypes(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['primary_use']= df['primary_use'].astype('category')
        df['meter'] = df["meter"].astype('category')
        df['site_id'] = df["site_id"].astype('category')
        df['building_id'] = df['building_id'].astype('category')
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [6]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [7]:
class CloudTimeCat(TransformerMixin):
        
    def transform(self, df, **transform_params):
        tempDf = df[['cloud_coverage', 'hour']].astype('int')
        tempDf['cloud_coverage'] = (tempDf['cloud_coverage']).astype('int')
        tempDf['hour'] = (tempDf['hour']).astype('int')
        tempDf = tempDf.astype('str')
        df['cloud_time_cat'] = 'c' + tempDf['cloud_coverage'] + 't' + tempDf['hour']
        df['cloud_time_cat'] = df['cloud_time_cat'].astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [8]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        return df.drop(self._drop_cols, axis=1)

    def fit(self, X, y=None, **fit_params):
        return self

In [9]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        year_built_median = df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [10]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [11]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [12]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [13]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [14]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [15]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [16]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [17]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [18]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)


gbm=LGBMRegressor(n_estimators=100, # for accuracy use large numbers like 6000 
                  learning_rate=0.23,
                  feature_fraction=0.9,
                  subsample=0.2,  # batches of 20% of the data
                  subsample_freq=1,
                  num_leaves=20,
                  metric='rmse',
                  verbose= 100)


In [19]:
# pre_a_pipes is for preprocessing that doesn't change impute
# values
pre_a_pipes = Pipeline(
    steps=[
        ('markNans',MarkNaNs()),
        ('convertToDatetime', ConvertToDatetime()),
        ('addRelativeHumidity',AddRelativeHumidity()),
        ('logSquareFeet', LogSquareFeet()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('setCatTypes', SetCatTypes()),
    ]
)

train_X = pre_a_pipes.transform(train.drop('meter_reading', axis=1))
test_X = pre_a_pipes.transform(test.drop(['row_id'], axis=1))
train_y = np.log1p(train['meter_reading'])

print(train_X.shape)
print(test_X.shape)
print(train_X.sample(n=20,  random_state=42))

del train, test
gc.collect()

(20216100, 28)
(41697600, 28)
         building_id meter           timestamp site_id  \
14245562        1324     1 2016-09-16 16:00:00      14   
1282718         1013     0 2016-01-24 06:00:00      10   
13883790         229     1 2016-09-10 07:00:00       2   
4781820          217     3 2016-04-01 01:00:00       2   
10415393        1434     0 2016-07-10 04:00:00      15   
1057008         1047     0 2016-01-20 04:00:00      12   
4507399          911     1 2016-03-26 20:00:00       9   
19478829        1039     0 2016-12-18 23:00:00      12   
8955615          265     0 2016-06-14 06:00:00       2   
13799839         896     0 2016-09-08 19:00:00       9   
15647011         973     0 2016-10-11 11:00:00       9   
2524294          813     0 2016-02-16 08:00:00       8   
10016102         870     0 2016-07-03 02:00:00       8   
3915750          898     0 2016-03-15 03:00:00       9   
17217526         903     0 2016-11-08 09:00:00       9   
11478           1442     2 2016-01-01 04:0

[20 rows x 28 columns]


28

In [20]:
# pre_b_pipes is for imputed values and final
# drops before modeling
drop_cols = ['timestamp', 'square_feet',
             'wind_direction',
             'precip_depth_1_hr', '_precip_depth_1_hr_nan',
             'year_built', '_year_built_nan', ]
fill_w_mean = ['floor_count','air_temperature','dew_temperature', 
              'precip_depth_1_hr', 'sea_level_pressure']
fill_w_zero = []


pre_b_pipes = Pipeline(
    steps=[
        ('fillMean',FillMean(fill_w_mean)),
        ('fillZeros',FillZeros(fill_w_zero)),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('dropClos', DropCols(drop_cols))
    ]
)

pipe = Pipeline(
    steps=[
        ('pre_b_pipes',pre_b_pipes),
        ('gbm', gbm)]
)



In [21]:
# Cross val testing - can be skipped
scores = cross_val_score(pipe, train_X, train_y, cv=5, 
                        scoring=rmsee_scorer)
print('rmsee scores:\n', scores)


rmsee scores:
 [-1.3405416  -1.26920667 -1.10308816 -1.20988481 -1.40712991]


In [22]:
del scores
gc.collect()

87

In [23]:
# Grid param search - can be skpped
grid_param = {
    #'n_estimators': [1000, 3000, 6000],
    'gbm__subsample': [0.1, 0.2]
}

gd_sr = GridSearchCV(pipe,
                     param_grid=grid_param,
                     scoring=rmsee_scorer,
                     cv=3,
                     n_jobs=-1)

gd_sr.fit(train_X, train_y)

best_parameters = gd_sr.best_params_
print(best_parameters)
best_result = gd_sr.best_score_
print(best_result)


{'gbm__subsample': 0.2}
-1.3840372480155383


In [24]:
del gd_sr
gc.collect

<function gc.collect(generation=2)>

In [25]:
# fit on all the data
pipe.fit(train_X, train_y, 
         gbm__eval_metric=rmsee, gbm__verbose=100)


Pipeline(memory=None,
         steps=[('pre_b_pipes',
                 Pipeline(memory=None,
                          steps=[('fillMean',
                                  <__main__.FillMean object at 0x0000019F45083BA8>),
                                 ('fillZeros',
                                  <__main__.FillZeros object at 0x0000019F45083C88>),
                                 ('imputeCloudCoverage',
                                  <__main__.ImputeCloudCoverage object at 0x0000019F48A947B8>),
                                 ('imputeYearBuilt',
                                  <__main__.ImputeYearBuilt object at 0x0000019F48494358>),
                                 ('dro...
                               colsample_bytree=1.0, feature_fraction=0.9,
                               importance_type='split', learning_rate=0.23,
                               max_depth=-1, metric='rmse',
                               min_child_samples=20, min_child_weight=0.001,
               

In [26]:
# Get features list for fit - can be skipped
imprtc_df = pd.DataFrame()
imprtc_df['feature'] = pre_b_pipes.transform(train_X).columns   
imprtc_df['importance'] = pipe.named_steps['gbm'].feature_importances_
imprtc_df.sort_values('importance', ascending=False, inplace= True)
print(imprtc_df)
print(imprtc_df.nsmallest(7,'importance')['feature'].tolist())



                    feature  importance
0               building_id         869
1                     meter         221
7           dew_temperature         184
5           air_temperature         147
8        sea_level_pressure          92
20                     hour          90
2                   site_id          71
17          log_square_feet          58
18                dayofweek          48
4               floor_count          23
12      _cloud_coverage_nan          23
6            cloud_coverage          21
9                wind_speed          18
21             building_age          12
3               primary_use          11
19                  weekday           7
14  _sea_level_pressure_nan           2
13     _dew_temperature_nan           1
10         _floor_count_nan           1
11     _air_temperature_nan           1
15      _wind_direction_nan           0
16          _wind_speed_nan           0
['_wind_direction_nan', '_wind_speed_nan', '_dew_temperature_nan', '_floor_count

In [ ]:
set_size = len(test_X)
iterations = 100
batch_size = set_size // iterations

print(set_size, iterations, batch_size)
assert set_size == iterations * batch_size

meter_reading = []
for i in tqdm(range(iterations)):
    pos = i*batch_size
    batch = np.expm1(pipe.predict(test_X.iloc[pos : pos+batch_size]).clip(0))
    meter_reading.extend(batch)

print(len(meter_reading))
assert len(meter_reading) == set_size

41697600 100 416976


 43%|██████████████████████████████████▊                                              | 43/100 [02:34<04:37,  4.87s/it]

In [ ]:
sub = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
# hack to prevent negative numbers
print(sub.shape)
sub['meter_reading'] = meter_reading
sub.to_csv('submission.csv', index = False)